In [1]:
from keras.layers import Dense, Activation, Embedding
from keras.preprocessing import sequence
from collections import defaultdict
from keras.models import Sequential
from keras.layers import LSTM,GRU,Bidirectional
from gensim import corpora
import numpy as np
import pickle,re

Using TensorFlow backend.
d:\installations\miniconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
stoplist = set('a is i of the am'.split())
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower())
    tokenized = [w for w in text.split() if w not in stoplist]
    return tokenized

In [3]:
docs,y= [],[]
with open('./sport.txt','r',encoding='latin-1') as csv:
	next(csv)
	for line in csv:
		text = line
		text = tokenizer(text)
		docs.append(text)
		y.append([1,0])

In [4]:
frequency = defaultdict(int)

for text in docs:
    for token in text:
        frequency[token] += 1

docs = [[token for token in text if frequency[token] >= 1] for text in docs]
dictionary = corpora.Dictionary(docs)
print(dictionary)

Dictionary(382 unique tokens: ['roger', 'eugenie', '10', 'raonic', 'australian']...)


In [5]:
val = dictionary.token2id
print(val['kohli'])

353


In [6]:
# for i in range(len(docs)):
#     for token in range(len(docs[i])):
#         t = docs[i][token]
#         docs[i][token] = val[t]

# with open('sport.pkl','wb') as f:
#     pickle.dump(docs, f)
pickle_in = open('sport.pkl','rb')
texts = pickle.load(pickle_in)  

In [7]:
maxlen = 25
texts = sequence.pad_sequences(texts, maxlen=maxlen,padding='post')
texts = np.array(texts)
X_train = texts[:50]
y_train = y[:50]
X_test = texts[50:]
y_test = y[50:]
print('X_train shape:', X_train.shape)


X_train shape: (50, 25)


In [11]:
batch_size = 2
max_features = 382
model = Sequential()
model.add(Embedding(max_features, 16))
model.add(LSTM(128,W_regularizer='l2', U_regularizer='l2', b_regularizer='l2'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

print('Training...')
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10)



Training...
Epoch 1/10
50/50 [==============================] - 3s - loss: 6.6815 - acc: 0.9600          

In [12]:
# pre = "virat kohli plays against england in first odi as India's new captain"
# pre = tokenizer(pre)
# for token in range(len(pre)):
#         t = pre[token]
#         pre[token] = val[t]
model.predict(pre)

array([[ 0.56966466,  0.43033537],
       [ 0.57086414,  0.42913577],
       [ 0.57024705,  0.42975298],
       [ 0.57296646,  0.42703357],
       [ 0.57163209,  0.42836797],
       [ 0.56947953,  0.43052045],
       [ 0.56923568,  0.43076432],
       [ 0.57045734,  0.42954272],
       [ 0.56962365,  0.43037635],
       [ 0.56987345,  0.43012658],
       [ 0.57095677,  0.42904317],
       [ 0.57039267,  0.42960733],
       [ 0.57025045,  0.42974955]], dtype=float32)